In [1]:
# import needed packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import math
import re
import time

In [173]:
# get id function
def get_id(term):
    id_list = []
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    url_search = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200"
    info = BeautifulSoup(requests.get(url_search).text, 'lxml')
    
    term_count = int(info.find('count').text)
    id_list += [single_term.text for single_term in info.find_all('id')]
    
    for page in range(1, math.ceil(term_count/200)):
        page_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200&retstart={page*200}"
        page_info = BeautifulSoup(requests.get(page_url).text, 'lxml')
        id_list += [single_term.text for single_term in page_info.find_all('id')]
    
    return id_list, term_count


In [178]:
term = "(PBMC)+AND+(single-cell)"
id_list, term_count = get_id(term=term)

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [179]:
len(id_list)

7151

In [180]:
# check result
print(len(set(id_list)) == term_count)
print(term_count)

True
7151


In [181]:
# reserve result
with open('/home/ouyangkang/Learning/Python/other_thing/term_search_id_result_11_5.txt', 'w') as f:
    f.writelines("\n".join(id_list))

In [2]:
with open('/home/ouyangkang/Learning/Python/other_thing/term_search_id_result_11_5.txt', 'r') as f:
    id_list = [i.strip('\n') for i in f.readlines()]

In [325]:
# define a function to get detailed info
def id_detailed(id_list): # the max capacity of id list is 20
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    return_list = []
    
    for page in range(math.ceil(len(id_list)/20)):
        id_search = ",".join(id_list[page*20:(page+1)*20])
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
                       "Accept-Encoding": "utf-8"}
        id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"
        id_info = BeautifulSoup(requests.get(id_url, headers=headers).text, 'html.parser')

        # Title
        title = [i.text for i in id_info.find_all('item', {'name': 'title'})]
        # Summary
        summary = [i.text for i in id_info.find_all('item', {'name': 'summary'})]
        # GSE
        accession_pattern = re.compile(r'</id>\n<item name="Accession" type="String">([^<]+)</item>')
        gse = [re.search(accession_pattern, str(id_info).split('<docsum>')[i]).group(1) for i in range(1, len(str(id_info).split('<docsum>')))]
        # Taxon
        taxon = [i.text for i in id_info.find_all('item', {'name': 'taxon'})]
        # sequencing type
        seq_type = [i.text for i in id_info.find_all('item', {'name': 'gdsType'})]
        # Publish date
        p_date = [i.text for i in id_info.find_all('item', {'name': 'PDAT'})]
        # not each batch urls are 20
        for num in range(len(id_list[page*20:(page+1)*20])):
            return_list.append([title[num], summary[num], gse[num], taxon[num], seq_type[num], p_date[num]])
        # print(f"the {page*20}-{(page+1)*20} ids have finished")
        # print(page)


    return return_list

In [321]:
page = 10
API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
id_search = ",".join(id_list[page*20:(page+1)*20])
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
                "Accept-Encoding": "utf-8"}
id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"
id_info = BeautifulSoup(requests.get(id_url, headers=headers).text, 'html.parser')

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [326]:
result_info = id_detailed(id_list=id_list)

In [327]:
# check length of each element
for i in range(len(result_info)):
    if len(result_info[i]) != 6:
        print(result_info[i])

print(len(result_info))

7151


In [349]:
# filter non-homo sapiens data and store result
no_filtering_result = pd.DataFrame(result_info, columns=['Title', 'Summary', 'GSE', 'Taxon', 'Sequencing_type', 'Publish_date'])
# avoid , impact csv format
no_filtering_result['Title'] = no_filtering_result['Title'].str.replace(',', '__')
no_filtering_result['Summary'] = no_filtering_result['Summary'].str.replace(',', '__')
# filtering
filtering_result = no_filtering_result[no_filtering_result['Taxon'].str.contains('Homo')]
filtering_result.to_csv('/home/ouyangkang/Learning/Python/other_thing/GEO_filtering_result_11_5.csv', index=False, encoding='UTF-8')

In [2]:
filtering_result = pd.read_csv('/home/ouyangkang/Learning/Python/other_thing/GEO_filtering_result_11_5.csv')

In [3]:
filtering_result.head()

,Title,Summary,GSE,Taxon,Sequencing_type,Publish_date
0,Factors contributing to Chimeric Antigen Recep...,Immune cell populations are composed of multip...,GSE187438,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/02
1,Pathophysiology of androgen-associated endothe...,Abundant evidence suggests that women with pol...,GSE246456,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/01
2,Immune landscape of PBMC based on single-cell ...,Although an increase in studies has revealed t...,GSE233315,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
3,Gene expression profile at single cell level o...,We used single cell RNA sequencing (scRNAseq) ...,GSE229626,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
4,Gene expression profiles of peripheral blood m...,An autoimmune B cell origin for childhood idio...,GSE233276,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/29


In [4]:
print(len(set(filtering_result['GSE'])))
gse_list = list(set(filtering_result.GSE.tolist()))
set(filtering_result["GSE"].tolist()) - (set(filtering_result[filtering_result['GSE'].str.contains("GSM")]["GSE"].tolist()) | set(filtering_result[filtering_result['GSE'].str.contains("GSE")]["GSE"].tolist()))

6968


{'GPL96'}

In [20]:
filtering_result[filtering_result.GSE.str.contains('GSE')]

,Title,Summary,GSE,Taxon,Sequencing_type,Publish_date
0,Factors contributing to Chimeric Antigen Recep...,Immune cell populations are composed of multip...,GSE187438,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/02
1,Pathophysiology of androgen-associated endothe...,Abundant evidence suggests that women with pol...,GSE246456,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/01
2,Immune landscape of PBMC based on single-cell ...,Although an increase in studies has revealed t...,GSE233315,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
3,Gene expression profile at single cell level o...,We used single cell RNA sequencing (scRNAseq) ...,GSE229626,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
4,Gene expression profiles of peripheral blood m...,An autoimmune B cell origin for childhood idio...,GSE233276,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/29
...,...,...,...,...,...,...
529,Comparison of haploid and diploid clones of KB...,The series were performed to study the changes...,GSE56465,Homo sapiens,Expression profiling by array,2014/04/10
530,Analysis of gene expression response of CLL ce...,In the marrow and lymphatic tissues__ chronic ...,GSE13811,Homo sapiens,Expression profiling by array,2008/12/11
531,Genetic alteration in recurrent melanoma,It is generally accepted that human cancers de...,GSE8777,Homo sapiens,Genome variation profiling by array,2008/07/22
532,SAGE and MPSS libraries from activated CD4+ T ...,SAGE and MPSS libraries were produced from the...,GSE8612,Homo sapiens,Expression profiling by SAGE; Expression profi...,2007/08/02


Because the results of web spiber include GSE, GSM and GPL causing it is a chanllege to construct an unique function to get same infomation, my strategy is using GPL and GSM to get GSE info

In [10]:
gse_list = filtering_result[filtering_result["GSE"].str.contains("GSE")]['GSE'].tolist()

In [15]:
def get_gse_info(gse_list):
    result_list = []
    for gse in gse_list:
        try:
            gse_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={gse}"
            headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"}
            gse_response = requests.get(gse_url, headers=headers,timeout=(30, 10))
            gse_response.encoding = 'utf-8'
            gse_info = BeautifulSoup(gse_response.text, 'html.parser')


            getting_info = gse_info.find_all('td', {"style": "text-align: justify"})
            # title
            title_info = getting_info[0].text
            # summary
            summary_info = getting_info[1].text
            # over design
            design_info = getting_info[2].text
            # organization name
            org_name = getting_info[3].text
            # country
            country = getting_info[-1].text
            # Experiment type
            exp_pattern = re.compile(r'<td>(.*)<br/></td>')
            exp_type = re.findall(exp_pattern, str(gse_info))[0]
            # contributor
            contributor = ";".join([i.text for i in gse_info.find_all('a', href=lambda value: value and '/pubmed/?term' in value)])
            # pmid
            try:
                pmid = gse_info.find('span', {'class': 'pubmed_id'}).text
            except:
                pmid = None
            # submission time and last update time
            time_pattern = re.compile(r'<td>([A-Z][a-z]+ \d{1,2}, \d{4})</td>')
            # first time is submission date, second last update date
            sub_time = re.findall(time_pattern, str(gse_info))[0]
            last_update_time = re.findall(time_pattern, str(gse_info))[1] 
            # platform
            platform = gse_info.find_all('td', {'valign':'top'})[1].text
            # sample
            gsm_list = []
            for i in gse_info.find_all('td', {'valign':'top'}):
                gsm = i.find('a', href=lambda value: value and '/geo/query/acc.cgi?acc' in value)
                if "GSM" in str(gsm):
                    gsm_list.append(gsm.get('href').strip('/geo/query/acc.cgi?acc='))
            sample_info = ";".join(gsm_list)
            # status
            statu_pattern = re.compile(r'<td>Public on (.*)</td>')
            statu = re.findall(statu_pattern, str(gse_info))[0]

            # "GSE", "Title", "PMID","Summary", "Design", "Experiment_type", "Platform", "Contributors", "Organization_name", "Country", "Sample", "Status", "Submission_date", "Last_update_date"
            result_list.append([gse, title_info, pmid, summary_info,design_info, exp_type, platform, contributor, org_name, country, sample_info, statu, sub_time, last_update_time])
            time.sleep(2)
        except:
            print(gse)
    return result_list

In [12]:
gse_result = get_gse_info(gse_list)

GSE161329
GSE147928


In [16]:
remaining_gse = ['GSE161329', 'GSE147928']
remaining_result = get_gse_info(remaining_gse)

In [18]:
for result in remaining_result:
    gse_result.append(result)

In [20]:
result_print = pd.DataFrame(gse_result, columns=["GSE", "Title", "PMID","Summary", "Design", "Experiment_type", "Platform", "Contributors", "Organization_name", "Country", "Sample", "Status", "Submission_date", "Last_update_date"])
result_print.to_csv('/home/ouyangkang/Learning/Python/other_thing/gse_detailed_result.csv', index=None)

In [21]:
result_print.head()

,GSE,Title,PMID,Summary,Design,Experiment_type,Platform,Contributors,Organization_name,Country,Sample,Status,Submission_date,Last_update_date
0,GSE187438,Factors contributing to Chimeric Antigen Recep...,None,Immune cell populations are composed of multip...,PBMC or BMMC from the patients' blood or bone ...,Expression profiling by high throughput sequen...,Illumina HiSeq 4000 (Homo sapiens),Li N;Rana T;Zhu Z,"University of California, San Diego",USA,GSM5668378;GSM5668379;GSM5668380;GSM5668381;GS...,"Nov 02, 2023","Nov 03, 2021","Nov 02, 2023"
1,GSE246456,Pathophysiology of androgen-associated endothe...,None,Abundant evidence suggests that women with pol...,single-cell RNA-seq,Expression profiling by high throughput sequen...,Illumina NextSeq 500 (Homo sapiens),Wu CE;Ho HN,National Taiwan University Hospital,Taiwan,GSM7869173;GSM7869174;GSM7869175;GSM7869176,"Nov 01, 2023","Oct 27, 2023","Nov 01, 2023"
2,GSE233315,Immune landscape of PBMC based on single-cell ...,None,Although an increase in studies has revealed t...,"In this case‒control study, 14 male and 6 fema...",Expression profiling by high throughput sequen...,Illumina NovaSeq 6000 (Homo sapiens),Zhang R;Liu X;Li R,The Second Hospital of Shanxi Medical University,China,GSM7422419;GSM7422420,"Oct 31, 2023","May 24, 2023","Oct 31, 2023"
3,GSE229626,Gene expression profile at single cell level o...,None,We used single cell RNA sequencing (scRNAseq) ...,Purified human T cells from six healthy donors...,Expression profiling by high throughput sequen...,NextSeq 2000 (Homo sapiens),Vantourout P;Eum J;Conde Poole M;Hayday TS;Lai...,Marengo Therapeutics,USA,GSM7169472;GSM7169473;GSM7169474;GSM7169475;GS...,"Oct 31, 2023","Apr 13, 2023","Oct 31, 2023"
4,GSE233276,Gene expression profiles of peripheral blood m...,None,An autoimmune B cell origin for childhood idio...,PBMC were isolated from the blood of four chil...,Expression profiling by high throughput sequen...,Illumina NovaSeq 6000 (Homo sapiens),Al-Aubodah T;Piccirillo CA;Takano T,McGill University,Canada,GSM7421512;GSM7421513;GSM7421514;GSM7421515;GS...,"Oct 29, 2023","May 23, 2023","Oct 30, 2023"


In [24]:
filtering_gsm = set(filtering_result[filtering_result["GSE"].str.contains("GSM")]['GSE'].tolist())

In [36]:
result_included_gsm = []
for gsm_list in result_print["Sample"].str.split(';').tolist():
    result_included_gsm += gsm_list

print(f"the count of repeated gsm_id of gse result is {len(result_included_gsm) - len(set(result_included_gsm))}")

remaining_gsm = list(set(filtering_result[filtering_result["GSE"].str.contains("GSM")]['GSE'].tolist()) - set(result_included_gsm))
print(f"the count of excluding gsm of gse result if {len(remaining_gsm)}")

the count of repeated gsm_id of gse result is 2910
the count of excluding gsm of gse result if 2855


In [60]:
def gsm_to_gse(gsm_list):
    result_info = []

    for gsm in gsm_list:
        try:
            gsm_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={gsm}"
            headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"}
            gsm_response = requests.get(gsm_url, headers=headers,timeout=(30, 10))
            gsm_response.encoding = 'utf-8'
            gsm_info = BeautifulSoup(gsm_response.text, 'html.parser')
            result_info += [i.text for i in gsm_info.find_all('td', {"valign": "top"}) if "GSE" in str(i)]
            time.sleep(1)
        except:
            print(gsm)

    return result_info

In [61]:
gse_from_gsm_list = gsm_to_gse(remaining_gsm)

GSM2842344


check finding GSM2842344 corresponding GSE106540, GSE106544

In [65]:
gse_from_gsm_filtering = list(set(gse_from_gsm_list))
set(gse_from_gsm_filtering) - set(gse_list)

set()

In [77]:
pmid_list = []
for pmid in result_print[~result_print['PMID'].isna()]["PMID"].str.split(','):
    pmid_list += pmid

pmid_list = list(set(pmid_list))

In [ ]:
# detailed infomation of lituraures
def get_literature_info(literature_list):
    result_info = []
    API_KEY = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
    try:
        for pmid in literature_list:
            pmid_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmid}&api_key={API_KEY}&rettype=medline&retmode=text'
            pmid_respond = requests.get(pmid_url, timeout=(15, 10))
            pmid_respond.encoding = 'utf-8'
            literature = str(BeautifulSoup(pmid_respond.text, 'html.parser'))

            # DOI
            if '[doi]' in literature:
                doi = literature.split('[doi]')[0].split(' - ')[-1].strip()
            else:
                doi = ""
            # title
            title = literature.split('AB  - ')[1].split(' - ')[0].replace("\n      ", "").split('\n')[0]
            # abstract
            abs = literature.split('AB  - ')[1].split(' - ')[0].replace("\n      ", "").split('\n')[0]
            # 1st author
            author = literature.split('FAU - ')[1].split('\n')[0]
            # 1st affiliation
            tmp_affiliation = literature.split('FAU - ')[1]
            if "AD  - " in tmp_affiliation:
                affiliation = tmp_affiliation.split('AD  - ')[1].replace("\n      ", "").strip('\n')
            else:
                affiliation = ""
            # Journel
            journel = literature.split('TA  - ')[1].split('\n')[0]
            # publication time
            publication_time = literature.split('SO  - ')[1].split(';')[0].split('. ')[1]
            if ':' in publication_time:
                publication_time = publication_time.split(':')[0]
    except:
        print()

In [84]:
pmid = pmid_list[0]
API_KEY = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
pmid_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmid}&api_key={API_KEY}&rettype=medline&retmode=text'#medline
pmid_respond = requests.get(pmid_url, timeout=(15, 10))

In [93]:
literature = str(BeautifulSoup(pmid_respond.text, 'html.parser'))

In [100]:
# abstract
literature.split('AB  - ')[1].split(' - ')[0].replace("\n      ", "").split('\n')[0]
# title
title = re.split(r'\n[A-Z]{2,3}\s', literature.split('TI  - ')[1])[0].replace("\n      ", "")
if '\n' in title:
    title = title.replace("\n      ", "")
print(title)
# DOI
if '[doi]' in literature:
    doi = literature.split('[doi]')[0].split(' - ')[-1].strip()
else:
    doi = ""
print(doi)
# 1st author
literature.split('FAU - ')[1].split('\n')[0]
# 
tmp_affiliation = literature.split('FAU - ')[1]
if "AD  - " in tmp_affiliation:
    affiliation = tmp_affiliation.split('AD  - ')[1].replace("\n      ", "").strip('\n')
else:
    affiliation = ""
print(affiliation)

literature.split('TA  - ')[1].split('\n')[0]

publication_time = literature.split('SO  - ')[1].split(';')[0].split('. ')[1]
if ':' in publication_time:
    publication_time = publication_time.split(':')[0]
print(publication_time)

A comprehensive platform for analyzing longitudinal multi-omics data.
10.1038/s41467-023-37432-w
Allen Institute for Immunology, Seattle, WA, 98109, USA. suhas.vasaikar@alleninstitute.org.
2023 Mar 27
